In [1]:
import numpy as np
import utilities as uti
from plyfile import PlyData, PlyElement

import sys
sys.path.insert(1, './Colors')
import realman_gaussian_color as rgc
import gradient_color as gdc
import gaussian_color as gc

curvarute_type = 'gauss'
cam = np.array([0,0,0])
height = 200
width = int(height*0.5)

In [2]:
from PIL import Image
def projectImage(data,direction=False):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    frame = frame + 255
    for i in range(height):
        for j in range(width):
            if not np.isnan(data[i][j]):
                color = gdc.getColor(value=data[i][j],lowest=-0.5,highest=0.55)
                frame[i][j] = np.array(color)
                # color = rgc.getColor(data[i][j])
                # frame[i][j] = np.array(color[0:3])
    img = Image.fromarray(frame, 'RGB')
    # if direction:
    #     img.save(direction)
    img.show()
    return frame

def computeAverageCurvature(data):
    curvatures = data[:,:,0]
    count = data[:,:,1]
    for i in range(height):
        for j in range(width):
            if not np.isnan(curvatures[i][j]):
                curvatures[i][j] = curvatures[i][j]/24
    return curvatures

In [3]:
def interpolate(raw,radius=1):
    data = raw.copy()
    h = len(data)
    w = len(data[0])
    for i in range(h):
        for j in range(w):
            if getBorder(i,j,data) and np.isnan(data[i][j]):
                subarray = data[i-radius:i+radius+1,j-radius:j+radius+1]
                if np.count_nonzero(~np.isnan(subarray))<1:
                    print("point :"+str(i)+","+str(j)+" is isolated!")
                else:
                    data[i][j] = np.nansum(subarray)/np.count_nonzero(~np.isnan(subarray))
    return data

def getBorder(x,y,data):
    row = data[x]
    col = data[:,y]
    top = [np.NaN,np.NaN]
    bot = [np.NaN,np.NaN]
    right = [np.NaN,np.NaN]
    left = [np.NaN,np.NaN]
    h = len(data)
    w = len(data[0])
    for i in range(h):
        if np.isnan(top[1]):
            top[1] = col[i]
            top[0] = i
        if np.isnan(bot[1]):
            bot[1] = col[h - 1 - i]
            bot[0] = h -1 - i
    for i in range(w):
        if np.isnan(left[1]):
            left[1] = row[i]
            left[0] = i
        if np.isnan(right[1]):
            right[1] = row[w - 1 - i]
            right[0] = w - 1 - i
    return (left[0]<y<right[0]) and (top[0]<x<bot[0])

## Init for real data
    def init(index):
        plydata = PlyData.read('./Mesh/results/jean normal/nonback_'+curvarute_type+'_pose_'+str(index)+'.ply')
        points = np.zeros((plydata['vertex'].count,2))
        values = plydata['vertex']['quality']
        points[:,0] = plydata['vertex']['x']
        points[:,1] = plydata['vertex']['y']
        center = [np.average(points[:,0]),np.average(points[:,1])]
        points[:,0] = np.around((points[:,0] - center[0])*100 + width/2)
        points[:,1] = np.around((points[:,1] - center[1])*100 + height/2)
        return points,values

In [4]:
## Init for simulation
    def init(index):
        dic = './Mesh/results/24 frames/fg'+str(index)+'.ply'
        points,values = uti.filterBack(dic)
        center = [np.average(points[:,0]),np.average(points[:,1])]
        points[:,0] = np.around((points[:,0] - center[0])*10 + width/2)       # axis x
        points[:,1] = np.around((points[:,1] - center[1])*10 + height/2 + 20)    # axix y and 20 is the space at bottom
        return points,values

In [5]:
# Main
data = np.zeros((height, width,2))
data[:,:,0] = None
for i in range(1,25):
    points,values = init(i)
    for i in range(len(values)):
        if np.isnan(data[height-int(points[i][1])][width-int(points[i][0])][0]):
            data[height-int(points[i][1])][width-int(points[i][0])][0] = values[i]
        else:
            data[height-int(points[i][1])][width-int(points[i][0])][0] += values[i]
        data[height-int(points[i][1])][width-int(points[i][0])][1] += 1
    # # interpolate before calc average
    # data[:,:,0] = interpolate(data[:,:,0],2)str()i+'',...eding

In [6]:
raw = computeAverageCurvature(data)
frame1 = projectImage(raw,'./Images/raw_image_24frames.png')

alter = interpolate(raw,2)
#alter = np.nan_to_num(alter)
frame2 = projectImage(alter,'./Images/gauss_sim_fixed_24frames.png')

    import pandas as pd
    df = pd.DataFrame(data=raw)
    alter2 = df.interpolate(method='nearest').to_numpy()
    frame3 = projectImage(alter2,'./Images/alter_interpolate_2.png')